In [1]:
# ISLR material, Original R to Python adaptation by Jordi Warmenhoven
# Subsequent Adaptation from http://www.science.smith.edu/~jcrouser/SDS293/
# Further simplifications and adaptions done above those.

### The Validation Set Approach

In [2]:
import pandas as pd
import numpy as np
import sklearn.linear_model as skl_lm
import matplotlib.pyplot as plt

In [3]:
df1 = pd.read_csv('Auto.csv', na_values='?').dropna()
df1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 392 entries, 0 to 396
Data columns (total 9 columns):
mpg             392 non-null float64
cylinders       392 non-null int64
displacement    392 non-null float64
horsepower      392 non-null float64
weight          392 non-null int64
acceleration    392 non-null float64
year            392 non-null int64
origin          392 non-null int64
name            392 non-null object
dtypes: float64(4), int64(4), object(1)
memory usage: 30.6+ KB


In [4]:
train_df = df1.sample(196, random_state = 1)
test_df = df1[~df1.isin(train_df)].dropna(how = 'all')

X_train = train_df['horsepower'].values.reshape(-1,1)
y_train = train_df['mpg']
X_test = test_df['horsepower'].values.reshape(-1,1)
y_test = test_df['mpg']

In [5]:
lm = skl_lm.LinearRegression()
model = lm.fit(X_train, y_train)

In [6]:
pred = model.predict(X_test)
from sklearn.metrics import mean_squared_error
MSE = mean_squared_error(y_test, pred)
print(MSE)

23.3619028926


In [7]:
from sklearn.preprocessing import PolynomialFeatures
# Quadratic
poly = PolynomialFeatures(degree=2)
X_train2 = poly.fit_transform(X_train)
X_test2 = poly.fit_transform(X_test)
model = lm.fit(X_train2, y_train)
print(mean_squared_error(y_test, model.predict(X_test2)))

# Cubic
poly = PolynomialFeatures(degree=3)
X_train3 = poly.fit_transform(X_train)
X_test3 = poly.fit_transform(X_test)
model = lm.fit(X_train3, y_train)
print(mean_squared_error(y_test, model.predict(X_test3)))

20.2526908583
20.325609366


In [8]:
train_df = df1.sample(196, random_state = 2)
test_df = df1[~df1.isin(train_df)].dropna(how = 'all')

X_train = train_df['horsepower'].values.reshape(-1,1)
y_train = train_df['mpg']
X_test = test_df['horsepower'].values.reshape(-1,1)
y_test = test_df['mpg']

# Linear
model = lm.fit(X_train, y_train)
print(mean_squared_error(y_test, model.predict(X_test)))

# Quadratic
poly = PolynomialFeatures(degree=2)
X_train2 = poly.fit_transform(X_train)
X_test2 = poly.fit_transform(X_test)
model = lm.fit(X_train2, y_train)
print(mean_squared_error(y_test, model.predict(X_test2)))

# Cubic
poly = PolynomialFeatures(degree=3)
X_train3 = poly.fit_transform(X_train)
X_test3 = poly.fit_transform(X_test)
model = lm.fit(X_train3, y_train)
print(mean_squared_error(y_test, model.predict(X_test3)))

25.1085390529
19.7225334705
19.9213678601


### Leave-One-Out Cross-Validation

In [9]:
model = lm.fit(X_train, y_train)
from sklearn.model_selection import cross_val_score, LeaveOneOut
loo = LeaveOneOut()
X = df1['horsepower'].values.reshape(-1,1)
y = df1['mpg'].values.reshape(-1,1)
loo.get_n_splits(X)

from sklearn.model_selection import KFold
crossvalidation = KFold(n_splits=392, random_state=None, shuffle=False)
scores = cross_val_score(model, X, y, scoring="neg_mean_squared_error", cv=crossvalidation,
 n_jobs=1)
print("Folds: " + str(len(scores)) + ", MSE: " + str(np.mean(np.abs(scores))) + ", STD: " + str(np.std(scores)))

Folds: 392, MSE: 24.2315135179, STD: 36.7973150364


In [10]:
for i in range(1,6):
    poly = PolynomialFeatures(degree=i)
    X_current = poly.fit_transform(X)
    model = lm.fit(X_current, y)
    scores = cross_val_score(model, X_current, y, scoring="neg_mean_squared_error", cv=crossvalidation,
 n_jobs=1)
    print("Degree-"+str(i)+" polynomial MSE: " + str(np.mean(np.abs(scores))) + ", STD: " + str(np.std(scores)))

Degree-1 polynomial MSE: 24.2315135179, STD: 36.7973150364
Degree-2 polynomial MSE: 19.2482131245, STD: 34.9984461518
Degree-3 polynomial MSE: 19.3349840641, STD: 35.765135678
Degree-4 polynomial MSE: 19.4244303091, STD: 35.6833527595
Degree-5 polynomial MSE: 19.0332073323, STD: 35.3173109734


### k-Fold Cross-Validation

In [11]:
crossvalidation = KFold(n_splits=10, random_state=1, shuffle=False)
for i in range(1,11):
    poly = PolynomialFeatures(degree=i)
    X_current = poly.fit_transform(X)
    model = lm.fit(X_current, y)
    scores = cross_val_score(model, X_current, y, scoring="neg_mean_squared_error", cv=crossvalidation,
 n_jobs=1)
    print("Degree-"+str(i)+" polynomial MSE: " + str(np.mean(np.abs(scores))) + ", STD: " + str(np.std(scores)))

Degree-1 polynomial MSE: 27.4399336523, STD: 14.5102507113
Degree-2 polynomial MSE: 21.2358400558, STD: 11.7973275289
Degree-3 polynomial MSE: 21.3366061835, STD: 11.8443397149
Degree-4 polynomial MSE: 21.3538869988, STD: 11.9863323388
Degree-5 polynomial MSE: 20.9056175409, STD: 12.1855513287
Degree-6 polynomial MSE: 20.7833453355, STD: 12.1652210683
Degree-7 polynomial MSE: 20.9534561267, STD: 12.0599965559
Degree-8 polynomial MSE: 21.0771389788, STD: 12.0444795097
Degree-9 polynomial MSE: 21.036758642, STD: 11.9482566282
Degree-10 polynomial MSE: 20.9869689716, STD: 11.8051164959


### An Application to Default Data

In [12]:
df2 = pd.read_csv('Default.csv', na_values='?').dropna()
df2.describe()

,Unnamed: 0,balance,income
count,10000.00000,10000.000000,10000.000000
mean,5000.50000,835.374886,33516.981876
std,2886.89568,483.714985,13336.639563
min,1.00000,0.000000,771.967729
25%,2500.75000,481.731105,21340.462903
50%,5000.50000,823.636973,34552.644802
75%,7500.25000,1166.308386,43807.729272
max,10000.00000,2654.322576,73554.233495


In [13]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10000 entries, 0 to 9999
Data columns (total 5 columns):
Unnamed: 0    10000 non-null int64
default       10000 non-null object
student       10000 non-null object
balance       10000 non-null float64
income        10000 non-null float64
dtypes: float64(2), int64(1), object(2)
memory usage: 468.8+ KB


In [14]:
import statsmodels.formula.api as smf
import statsmodels.api as sm
from sklearn.metrics import confusion_matrix, classification_report
for i in range(1,11):
    train_df2 = df2.sample(8000, random_state = i)
    test_df2 = df2[~df2.isin(train_df2)].dropna(how = 'all')
    model = smf.glm('default~balance', data=train_df2, family=sm.families.Binomial())
    result = model.fit()
    predictions_nominal = [ "Yes" if x < 0.5 else "No" for x in result.predict(test_df2)]
    print("----------------")
    print("Random Seed = " + str(i) + "")
    print("----------------")
    print(confusion_matrix(test_df2["default"], 
                       predictions_nominal))
    print(classification_report(test_df2["default"], predictions_nominal, digits = 3))
    print()

C:\Apps\Dir\Anaconda3\lib\site-packages\statsmodels\compat\pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


----------------
Random Seed = 1
----------------
[[1921    6]
 [  50   23]]
             precision    recall  f1-score   support

         No      0.975     0.997     0.986      1927
        Yes      0.793     0.315     0.451        73

avg / total      0.968     0.972     0.966      2000


----------------
Random Seed = 2
----------------
[[1919   13]
 [  47   21]]
             precision    recall  f1-score   support

         No      0.976     0.993     0.985      1932
        Yes      0.618     0.309     0.412        68

avg / total      0.964     0.970     0.965      2000


----------------
Random Seed = 3
----------------
[[1918   14]
 [  49   19]]
             precision    recall  f1-score   support

         No      0.975     0.993     0.984      1932
        Yes      0.576     0.279     0.376        68

avg / total      0.962     0.969     0.963      2000


----------------
Random Seed = 4
----------------
[[1932    4]
 [  46   18]]
             precision    recall  f1-score  

### The Bootstrap

In [15]:
portfolio_df = pd.read_csv('Portfolio.csv')
portfolio_df.head()

,X,Y
0,-0.895251,-0.234924
1,-1.562454,-0.885176
2,-0.417090,0.271888
3,1.044356,-0.734198
4,-0.315568,0.841983


In [16]:
def alpha(X,Y):
    return ((np.var(Y)-np.cov(X,Y))/(np.var(X)+np.var(Y)-2*np.cov(X,Y)))

In [17]:
X = portfolio_df.X[0:100]
y = portfolio_df.Y[0:100]
print(alpha(X,y))

[[ 1.07270947  0.57665115]
 [ 0.57665115  0.06414064]]


In [18]:
dfsample = portfolio_df.sample(frac=1, replace=True)
X = dfsample.X[0:100]
y = dfsample.Y[0:100]
print(alpha(X,y))

[[ 1.03847759  0.60113584]
 [ 0.60113584  0.04198121]]


In [19]:
def bstrap(df):
    tresult = 0
    for i in range(0,1000):
        dfsample = df.sample(frac=1, replace=True)
        X = dfsample.X[0:100]
        y = dfsample.Y[0:100]
        result = alpha(X,y)
        tresult += result
    fresult = tresult / 1000
    print(fresult)
bstrap(portfolio_df)

[[ 0.96729148  0.57771193]
 [ 0.57771193  0.09902743]]


### Estimating the Accuracy of a Linear Regression Model

In [20]:
from sklearn.utils import resample
auto_df = pd.read_csv('Auto.csv', na_values='?').dropna()
auto_df.describe()

,mpg,cylinders,displacement,horsepower,weight,acceleration,year,origin
count,392.000000,392.000000,392.000000,392.000000,392.000000,392.000000,392.000000,392.000000
mean,23.445918,5.471939,194.411990,104.469388,2977.584184,15.541327,75.979592,1.576531
std,7.805007,1.705783,104.644004,38.491160,849.402560,2.758864,3.683737,0.805518
min,9.000000,3.000000,68.000000,46.000000,1613.000000,8.000000,70.000000,1.000000
25%,17.000000,4.000000,105.000000,75.000000,2225.250000,13.775000,73.000000,1.000000
50%,22.750000,4.000000,151.000000,93.500000,2803.500000,15.500000,76.000000,1.000000
75%,29.000000,8.000000,275.750000,126.000000,3614.750000,17.025000,79.000000,2.000000
max,46.600000,8.000000,455.000000,230.000000,5140.000000,24.800000,82.000000,3.000000


In [21]:
auto_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 392 entries, 0 to 396
Data columns (total 9 columns):
mpg             392 non-null float64
cylinders       392 non-null int64
displacement    392 non-null float64
horsepower      392 non-null float64
weight          392 non-null int64
acceleration    392 non-null float64
year            392 non-null int64
origin          392 non-null int64
name            392 non-null object
dtypes: float64(4), int64(4), object(1)
memory usage: 30.6+ KB


In [22]:
lm = skl_lm.LinearRegression()
X = auto_df['horsepower'].values.reshape(-1,1)
y = auto_df['mpg']
clf = lm.fit(X,y)
print(clf.coef_, clf.intercept_)

[-0.15784473] 39.9358610212


In [23]:
from sklearn.metrics import mean_squared_error
Xsamp, ysamp = resample(X, y, n_samples=1000)
clf = lm.fit(Xsamp,ysamp)
print('Intercept: ' + str(clf.intercept_) + " Coef: " + str(clf.coef_))

Intercept: 39.964915632 Coef: [-0.15870295]
